In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

In [4]:
# read in relevant tables, change index to 1-index to match dataset
# i'm also printing the first row of each DataFrame as an easy reference point to columns

races = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/refs/heads/main/data/2021/2021-09-07/races.csv")
races.index = races.index + 1
print("races")
display(races.head(1))

drivers = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/refs/heads/main/data/2021/2021-09-07/drivers.csv")
drivers.index = drivers.index + 1
print("drivers")
display(drivers.head(1))

driver_standings = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/refs/heads/main/data/2021/2021-09-07/driver_standings.csv")
driver_standings.index = driver_standings.index + 1
print("driver_standings")
display(driver_standings.head(1))

constructors = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/refs/heads/main/data/2021/2021-09-07/constructors.csv")
constructors.index = constructors.index + 1
print("constructors")
display(constructors.head(1))

constructor_standings = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/refs/heads/main/data/2021/2021-09-07/constructor_standings.csv")
constructor_standings.index = constructor_standings.index + 1
print("constructor_standings")
display(constructor_standings.head(1))

results = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/refs/heads/main/data/2021/2021-09-07/results.csv")
results.index = results.index + 1
print("results")
display(results.head(1))

races


,raceId,year,round,circuitId,name,date,time,url
1,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...


drivers


,driverId,driverRef,number,code,forename,surname,dob,nationality,url
1,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton


driver_standings


...

,constructorId,constructorRef,name,nationality,url
1,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren


constructor_standings


,constructorStandingsId,raceId,constructorId,points,position,positionText,wins
1,1,18,1,14.0,1,1,1


results


,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
1,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1


In [99]:
# merge name columns for later merging into tables
drivers["driverName"] = drivers["forename"] + " " + drivers["surname"]

# rename columns for same reason
driver_standings.rename(columns={"position": "driverPosition", "points": "driverPoints"}, inplace=True)
constructors.rename(columns={"name": "constructorName"}, inplace=True)
constructor_standings.rename(columns={"position": "constructorPosition", "points": "constructorPoints"}, 
                             inplace=True)

# set \N value to NaN in results
results["position"] = pd.to_numeric(results["position"], errors="coerce")
results["position"] = results["position"].fillna(26)
results["position"] = results["position"].astype(int)

In [6]:
# I'm going to focus on data from the year that Schumacher started racing in F1
years = races[races["year"] >= 1980]["year"].unique()
last_raceId_per_year = races.groupby("year")["raceId"].max().reindex(years).to_numpy()
last_raceId_per_year

array([  17,   35,   52,   70,   89,  107,  123,  140,  157,  174,  190,
        206,  223,  239,  256,  272,  288,  304,  320,  336,  355,  371,
        387,  403,  419,  435,  451,  466,  482,  497,  511,  859,  879,
        899,  918,  945,  968,  988, 1009, 1030, 1047, 1073])

In [7]:
constructor_standings.head()

,constructorStandingsId,raceId,constructorId,constructorPoints,constructorPosition,positionText,wins
1,1,18,1,14.0,1,1,1
2,2,18,2,8.0,3,3,0
3,3,18,3,9.0,2,2,0
4,4,18,4,5.0,4,4,0
5,5,18,5,2.0,5,5,0


In [8]:
# remove all rows from `races` where 'raceId' not in `last_raceId_per_year`
filtered_races = races[races["raceId"].isin(last_raceId_per_year)]

# merge `driver_standings` stats with `filtered_races`, use inner join to only retain relevant rows
driver_details = pd.merge(
    driver_standings, filtered_races[["raceId", "year"]],
    on="raceId", how="inner"
)

# add 'constructorId' from `results`
driver_results = pd.merge(
    driver_details, results[["raceId", "driverId", "constructorId"]],
    on=["raceId", "driverId"], how="inner"
)

# add 'driverName' by merging with `drivers`
driver_results = pd.merge(
    driver_results, drivers[["driverId", "driverName"]],
    on="driverId", how="inner"
)

# add constructorPoints and constructorPosition and merge with constructor standings
driver_constructor_details = pd.merge(
    driver_results,
    constructor_standings[["raceId", "constructorId", "constructorPosition", "constructorPoints"]],
    on=["raceId", "constructorId"], how="inner"
)

# finally, add constructorName
final_table = pd.merge(
    driver_constructor_details,
    constructors[["constructorId", "constructorName"]],
    on="constructorId", how="inner"
)

# select and rename columns
# final_table = final_table[['driverName', 'year', 'position', 'constructorName', 'constructorPosition']]
final_table.drop(columns=["positionText"], inplace=True)

final_table[final_table["year"] == 2008]

,driverStandingsId,raceId,driverId,driverPoints,driverPosition,wins,year,constructorId,driverName,constructorPosition,constructorPoints,constructorName
0,355,35,1,98.0,1,5,2008,1,Lewis Hamilton,2,151.0,McLaren
1,356,35,2,60.0,6,0,2008,2,Nick Heidfeld,3,135.0,BMW Sauber
2,357,35,3,17.0,13,0,2008,3,Nico Rosberg,8,26.0,Williams
3,358,35,4,61.0,5,2,2008,4,Fernando Alonso,4,80.0,Renault
4,359,35,5,53.0,7,1,2008,1,Heikki Kovalainen,2,151.0,McLaren
5,360,35,6,9.0,15,0,2008,3,Kazuki Nakajima,8,26.0,Williams
6,361,35,7,4.0,17,0,2008,5,Sébastien Bourdais,6,39.0,Toro Rosso
7,362,35,8,75.0,3,2,2008,6,Kimi Räikkönen,1,172.0,Ferrari
8,363,35,9,75.0,4,1,2008,2,Robert Kubica,3,135.0,BMW Sauber
9,364,35,15,31.0,9,0,2008,7,Jarno Trulli,5,56.0,Toyota


In [9]:
# further cleanup of data

# sort by raceId and points to assign positions within each race
final_table.sort_values(by=["raceId", "driverPoints", "constructorPosition"], 
                        ascending=[True, False, True], inplace=True)

# recreate driverPosition based on sorted points (driver with max points gets position 1)
# note that those with 0 points have been ordered by myself according to their constructor's position
final_table["driverPosition"] = final_table.groupby("raceId").cumcount() + 1

# remove drivers who never scored a point (for diagram clarity)
drivers_score = final_table.groupby("driverId")["driverPoints"].sum()

drivers_who_score = drivers_score[drivers_score > 0].index
final_table = final_table[final_table["driverId"].isin(drivers_who_score)]

final_table[final_table["year"] == 2002]

,driverStandingsId,raceId,driverId,driverPoints,driverPosition,wins,year,constructorId,driverName,constructorPosition,constructorPoints,constructorName
141,12086,140,30,144.0,1,11,2002,6,Michael Schumacher,1,221.0,Ferrari
124,12104,140,22,77.0,2,4,2002,6,Rubens Barrichello,1,221.0,Ferrari
140,12087,140,31,50.0,3,0,2002,3,Juan Pablo Montoya,2,92.0,Williams
133,12094,140,23,42.0,4,1,2002,3,Ralf Schumacher,2,92.0,Williams
125,12103,140,14,41.0,5,1,2002,1,David Coulthard,3,65.0,McLaren
139,12088,140,8,24.0,6,0,2002,1,Kimi Räikkönen,3,65.0,McLaren
132,12095,140,18,14.0,7,0,2002,4,Jenson Button,4,23.0,Renault
123,12105,140,15,9.0,8,0,2002,4,Jarno Trulli,4,23.0,Renault
138,12089,140,56,8.0,9,0,2002,19,Eddie Irvine,7,8.0,Jaguar
131,12096,140,2,7.0,10,0,2002,15,Nick Heidfeld,5,11.0,Sauber


In [15]:
final_table.isna().sum()

driverStandingsId      0
raceId                 0
driverId               0
driverPoints           0
driverPosition         0
wins                   0
year                   0
constructorId          0
driverName             0
constructorPosition    0
constructorPoints      0
constructorName        0
dtype: int64

In [16]:
final_table.to_csv("f1_results_clean.csv")

In [11]:
final_table[(final_table["year"] < 1987) & (final_table["constructorPosition"] == 1)]

,driverStandingsId,raceId,driverId,driverPoints,driverPosition,wins,year,constructorId,driverName,constructorPosition,constructorPoints,constructorName
550,20425,419,95,70.0,2,5,1986,3,Nigel Mansell,1,141.0,Williams
529,20401,419,137,69.0,3,4,1986,3,Nelson Piquet,1,141.0,Williams
555,20974,435,117,73.0,1,5,1985,1,Alain Prost,1,90.0,McLaren
569,20991,435,182,14.0,10,1,1985,1,Niki Lauda,1,90.0,McLaren
591,21535,451,182,72.0,1,5,1984,1,Niki Lauda,1,143.5,McLaren
580,21524,451,117,71.5,2,7,1984,1,Alain Prost,1,143.5,McLaren
614,22046,466,163,49.0,3,3,1983,6,René Arnoux,1,89.0,Ferrari
610,22041,466,175,40.0,4,1,1983,6,Patrick Tambay,1,89.0,Ferrari
638,22731,482,175,25.0,6,1,1982,6,Patrick Tambay,1,74.0,Ferrari
639,22726,482,207,4.0,16,0,1982,6,Mario Andretti,1,74.0,Ferrari


In [100]:
# creating a separate table for individual races
new_races = races[races["year"] >= 1980]

# add year
all_driver_details = pd.merge(
    driver_standings, new_races[["raceId", "year"]],
    on="raceId", how="inner"
)

# add 'driverName' by merging with `drivers`
all_driver_results = pd.merge(
    all_driver_details, drivers[["driverId", "driverName"]],
    on="driverId", how="inner"
)

all_driver_results = pd.merge(
    all_driver_results, results[["driverId", "raceId", "position"]],
    on=["driverId", "raceId"], how="inner"
)

In [101]:
# add a column that adds what number race the current one is that year
all_driver_results.sort_values(["year", "raceId"], inplace=True)

for yr in years:
    i = 1
    uniq_raceIds = all_driver_results[all_driver_results["year"] == yr]["raceId"].unique()
    for rId in uniq_raceIds:
        all_driver_results.loc[all_driver_results["raceId"] == rId, "race_number"] = i
        
        i = i + 1


In [102]:
for col in all_driver_results.columns:
    all_driver_results.rename(columns={col: f"race_{col}"}, inplace=True)

all_driver_results.head()

,race_driverStandingsId,race_raceId,race_driverId,race_driverPoints,race_driverPosition,race_positionText,race_wins,race_year,race_driverName,race_position,race_race_number
11436,58952,498,158,0.0,28,28,0,1980,Eddie Cheever,26,1.0
11437,58951,498,136,0.0,27,27,0,1980,Jan Lammers,26,1.0
11438,58950,498,140,0.0,26,26,0,1980,Stefan Johansson,26,1.0
11439,58949,498,225,0.0,25,25,0,1980,Dave Kennedy,26,1.0
11440,58948,498,197,0.0,24,24,0,1980,Jean-Pierre Jarier,26,1.0


In [107]:
all_driver_results.sort_values(["race_race_number", "race_position"], inplace=True)

In [104]:
all_driver_results.to_csv("f1_race_details.csv")

In [105]:
all_driver_results.isna().sum()
all_driver_results[all_driver_results["race_year"] == 2014]

,race_driverStandingsId,race_raceId,race_driverId,race_driverPoints,race_driverPosition,race_positionText,race_wins,race_year,race_driverName,race_position,race_race_number
13758,66613,900,3,25.0,1,1,1,2014,Nico Rosberg,1,1.0
13743,66614,900,825,18.0,2,2,0,2014,Kevin Magnussen,2,1.0
13744,66615,900,18,15.0,3,3,0,2014,Jenson Button,3,1.0
13745,66616,900,4,12.0,4,4,0,2014,Fernando Alonso,4,1.0
13746,66617,900,822,10.0,5,5,0,2014,Valtteri Bottas,5,1.0
...,...,...,...,...,...,...,...,...,...,...,...
14135,67026,918,16,0.0,18,18,0,2014,Adrian Sutil,16,19.0
14144,67039,918,829,0.0,23,23,0,2014,Will Stevens,17,19.0
14133,67024,918,826,8.0,15,15,0,2014,Daniil Kvyat,26,19.0
14138,67031,918,813,2.0,16,16,0,2014,Pastor Maldonado,26,19.0


In [108]:
all_driver_results["race_position"].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 26, 19, 20, 21, 22, 23, 24])